In [1]:
import sys

sys.path.insert(0, "../src/")

In [2]:
import hail as hl
import numpy as np
import pyspark.sql.functions as f
from gentropy.common.session import Session
from gentropy.dataset.study_index import StudyIndex
from gentropy.dataset.study_locus import StudyLocus
from gentropy.dataset.summary_statistics import SummaryStatistics
from gentropy.datasource.gnomad.ld import GnomADLDMatrix

hl.stop()
session = Session(hail_home="/Users/dc16/Library/Caches/pypoetry/virtualenvs/otgenetics-8ghCgvjI-py3.10/lib/python3.10/site-packages/hail",
                  extended_spark_conf={
        "spark.jars": "/Users/dc16/Library/Caches/pypoetry/virtualenvs/otgenetics-8ghCgvjI-py3.10/lib/python3.10/site-packages/hail/backend/hail-all-spark.jar",
        "spark.driver.extraClassPath": "/Users/dc16/Library/Caches/pypoetry/virtualenvs/otgenetics-8ghCgvjI-py3.10/lib/python3.10/site-packages/hail/backend/hail-all-spark.jar",
        "spark.executor.extraClassPath": "./hail-all-spark.jar",
        "spark.serializer": "org.apache.spark.serializer.KryoSerializer",
        "spark.kryo.registrator": "is.hail.kryo.HailKryoRegistrator",
        "spark.kryoserializer.buffer.max": "1g",
        "spark.executor.memory": "5g",
        "spark.driver.memory": "5g",
        }
)
hl.init(sc=session.spark.sparkContext, log="dev/null")

Loading BokehJS ...

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/Users/dc16/Library/Caches/pypoetry/virtualenvs/otgenetics-8ghCgvjI-py3.10/lib/python3.10/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-sp

In [3]:
sl_path = "gs://finngen_data/r10/study_locus_datasets/finngen_summary_statistics_window_clumped"

sl = StudyLocus.from_parquet(session, sl_path)
sl.df = sl.df.sample(False, 0.1).limit(1)
study = sl.df.select("studyId").collect()[0][0]
sl.df.show()
study

+--------------------+---------------+----------+--------+------+--------------------+-------+--------------+--------------+-------------------------------+-------------+-------------------+---------------+-----------------+----------------+------------------+----------+-----+-----+
|        studyLocusId|      variantId|chromosome|position|region|             studyId|   beta|pValueMantissa|pValueExponent|effectAlleleFrequencyFromSource|standardError|subStudyDescription|qualityControls|finemappingMethod|credibleSetIndex|credibleSetlog10BF|sampleSize|ldSet|locus|
+--------------------+---------------+----------+--------+------+--------------------+-------+--------------+--------------+-------------------------------+-------------+-------------------+---------------+-----------------+----------------+------------------+----------+-----+-----+
|-7893241189694299986|12_49942262_A_C|        12|49942262|  null|FINNGEN_R10_AB1_M...|1.23452|         1.297|            -9|                     6.9

'FINNGEN_R10_AB1_MYCOSES'

In [18]:
finngen_study_index = StudyIndex.from_parquet(session, "gs://finngen_data/r10/study_index")

In [5]:
sumstats = SummaryStatistics.get_locus_sumstats(session=session, window=1_000_000, locus=sl.df)

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/Users/dc16/Library/Caches/pypoetry/virtualenvs/otgenetics-8ghCgvjI-py3.10/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/dc16/Library/Caches/pypoetry/virtualenvs/otgenetics-8ghCgvjI-py3.10/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/dc16/.pyenv/versions/3.10.8/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
sumstats.show()

+--------------------+--------------+----------+--------+-----------+----------+--------------+--------------+-------------------------------+-------------+
|             studyId|     variantId|chromosome|position|       beta|sampleSize|pValueMantissa|pValueExponent|effectAlleleFrequencyFromSource|standardError|
+--------------------+--------------+----------+--------+-----------+----------+--------------+--------------+-------------------------------+-------------+
|FINNGEN_R10_ANTID...|2_62590843_T_C|         2|62590843|  0.0123876|      null|         2.172|            -1|                       0.137061|     0.010039|
|FINNGEN_R10_ANTID...|2_62591132_G_C|         2|62591132| -0.0127415|      null|          1.01|            -1|                       0.728786|   0.00776884|
|FINNGEN_R10_ANTID...|2_62591320_C_T|         2|62591320| -0.0862858|      null|         2.013|            -1|                     0.00271666|    0.0675235|
|FINNGEN_R10_ANTID...|2_62591529_A_G|         2|62591529| 

In [ ]:
sumstats = SummaryStatistics.from_parquet(session, f"gs://gwas_catalog_data/harmonised_summary_statistics/{study}.parquet")
sumstats.df.show()

In [123]:
study_index = StudyIndex.from_parquet(session, "/Users/dc16/data/study_index/", recursiveFileLookup=True)
study_index.df.filter(f.col("studyId") == study).select("studyId", "hasSumstats").show()

+----------+-----------+
|   studyId|hasSumstats|
+----------+-----------+
|GCST007432|       true|
+----------+-----------+



In [4]:
locus_index = GnomADLDMatrix.get_locus_index(study_locus_row=sl.df, session=session, window_size=1_000_000)
locus_matrix = GnomADLDMatrix.get_locus_matrix(locus_index, gnomad_ancestry="nfe")
locus_matrix[0:5, 0:5]

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 8) / 8]
Traceback (most recent call last):
  File "/Users/dc16/Library/Caches/pypoetry/virtualenvs/otgenetics-8ghCgvjI-py3.10/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/dc16/Library/Caches/pypoetry/virtualenvs/otgenetics-8ghCgvjI-py3.10/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/dc16/.pyenv/versions/3.10.8/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)
